# 🫁 Chest X-ray Disease Detection System

## Using Pre-trained Models + Gradio Web App

**Author**: Muhammad Saad Khan  
**Date**: October 30, 2025

---

## ? What This Notebook Does:

### Part 1: Setup & Configuration

1. ✅ Mount Google Drive and set up project directory
2. ✅ Ensure required packages are installed

### Part 2: Web App (Instant!)

3. ✅ Load pre-trained Pneumonia and COVID-19 models
4. ✅ Launch Gradio web interface
5. ✅ Upload X-rays and get predictions
6. ✅ Beautiful UI with confidence scores
7. ✅ Shareable public link!

---

## 🚀 Quick Start:

1. **Enable GPU**: Runtime → Change runtime type → GPU
2. **Ensure models are in Google Drive** (see Model Locations section)
3. **Run all cells** in order
4. **Get shareable link** to your app!

---


## ⚙️ PART 0: Setup & Configuration


In [1]:
# Check GPU availability
import tensorflow as tf
print("🔍 System Check:")
print("=" * 60)
print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {len(tf.config.list_physical_devices('GPU')) > 0}")
print(f"GPU Devices: {tf.config.list_physical_devices('GPU')}")
print("=" * 60)

if not tf.config.list_physical_devices('GPU'):
    print("\n⚠️ WARNING: No GPU detected!")
    print("Go to: Runtime → Change runtime type → GPU")
else:
    print("\n✅ GPU is ready!")

🔍 System Check:
TensorFlow version: 2.19.0
GPU Available: False
GPU Devices: []

⚠️ WARNING: No GPU detected!
Go to: Runtime → Change runtime type → GPU


In [2]:
# Install required packages
print("📦 Installing required packages...")
!pip install -q gradio opencv-python-headless scikit-learn pillow matplotlib seaborn
print("✅ Packages installed!")

📦 Installing required packages...
✅ Packages installed!


# 🫁 Chest X-ray Disease Detection System

## Using Pre-trained Models + Gradio Web App

**Author**: Muhammad Saad Khan  
**Date**: October 30, 2025

---

## ? What This Notebook Does:

### Part 1: Setup & Configuration

1. ✅ Mount Google Drive and set up project directory
2. ✅ Ensure required packages are installed

### Part 2: Web App (Instant!)

3. ✅ Load pre-trained Pneumonia and COVID-19 models
4. ✅ Launch Gradio web interface
5. ✅ Upload X-rays and get predictions
6. ✅ Beautiful UI with confidence scores
7. ✅ Shareable public link!

---

## 🚀 Quick Start:

1. **Enable GPU**: Runtime → Change runtime type → GPU
2. **Ensure models are in Google Drive** (see Model Locations section)
3. **Run all cells** in order
4. **Get shareable link** to your app!

---


In [ ]:
# Import all libraries
import gradio as gr
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras import layers, models
from tensorflow import keras
import tensorflow as tf
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from PIL import Image
import warnings
warnings.filterwarnings('ignore')


print("✅ All imports successful!")

✅ All imports successful!


## 📁 PART 1: Mount Drive & Setup Project


In [4]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

print("\n✅ Google Drive mounted!")

Mounted at /content/drive

✅ Google Drive mounted!


In [5]:
# Set up project directories
# IMPORTANT: Update this path to match your Google Drive structure!
PROJECT_DIR = '/content/drive/MyDrive/Zikriya'

# Create directories if they don't exist
os.makedirs(f'{PROJECT_DIR}/models', exist_ok=True)
os.makedirs(f'{PROJECT_DIR}/results', exist_ok=True)

print(f"📂 Project directory: {PROJECT_DIR}")
print("\n📊 Available datasets:")
!ls -la {PROJECT_DIR}/datasets/

📂 Project directory: /content/drive/MyDrive/Zikriya

📊 Available datasets:
total 12
drwx------ 2 root root 4096 Oct 29 17:15 covid_organized
drwx------ 2 root root 4096 Oct 29 17:15 lung_organized
drwx------ 2 root root 4096 Oct 29 17:10 pneumonia


In [6]:
# Define dataset paths
# IMPORTANT: Update this path to match your Google Drive structure!
PROJECT_DIR = '/content/drive/MyDrive/Zikriya'

print("✅ Project directory configured:")
print(f"   Project: {PROJECT_DIR}")

✅ Project directory configured:
   Project: /content/drive/MyDrive/Zikriya


## 🤖 PART 2: Load Models & Build Gradio App


In [ ]:
# Load trained models
print("📦 Loading trained models...")

try:
    # IMPORTANT: Update these paths to your .h5 model locations!
    pneumonia_model = tf.keras.models.load_model(
        # Assuming the .h5 file is named like this
        f'{PROJECT_DIR}/models/pneumonia_model_best.h5'
    )
    print("✅ Pneumonia model loaded")
except Exception as e:
    print(f"⚠️ Error loading Pneumonia model: {e}")
    print("Please ensure the path to your .h5 model is correct.")


try:
    # IMPORTANT: Update these paths to your .h5 model locations!
    covid_model = tf.keras.models.load_model(
        # Assuming the .h5 file is named like this
        f'{PROJECT_DIR}/models/covid_model_best.h5'
    )
    print("✅ COVID-19 model loaded")
except Exception as e:
    print(f"⚠️ Error loading COVID-19 model: {e}")
    print("Please ensure the path to your .h5 model is correct.")

📦 Loading trained models...


✅ Pneumonia model loaded


✅ COVID-19 model loaded


In [14]:
def preprocess_image(image):
    """
    Preprocess image for prediction.
    """
    # Convert to RGB if needed
    if image.mode != 'RGB':
        image = image.convert('RGB')

    # Resize
    image = image.resize((224, 224))

    # Convert to array and normalize
    img_array = np.array(image) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    return img_array

In [ ]:
def predict_disease(image):
    """
    Main prediction function for Gradio.
    """
    if image is None:
        return "Please upload an X-ray image!", {}, None

    # Preprocess
    img = Image.fromarray(image.astype('uint8'), 'RGB')
    processed = preprocess_image(img)

    # Predict with both models
    pneumonia_pred = pneumonia_model.predict(processed, verbose=0)[0][0]
    covid_pred = covid_model.predict(processed, verbose=0)[0][0]

    # Create results
    results = {
        "Normal (Pneumonia Model)": float(1 - pneumonia_pred),
        "Pneumonia": float(pneumonia_pred),
        "Normal (COVID Model)": float(1 - covid_pred),
        "COVID-19": float(covid_pred)
    }

    # Determine overall status
    max_disease = max(pneumonia_pred, covid_pred)

    if max_disease > 0.7:
        if pneumonia_pred > covid_pred:
            status = f"⚠️ **PNEUMONIA DETECTED** (Confidence: {pneumonia_pred*100:.1f}%)"
        else:
            status = f"⚠️ **COVID-19 DETECTED** (Confidence: {covid_pred*100:.1f}%)"
    elif max_disease > 0.5:
        status = "⚠️ Possible abnormality detected - consult a doctor"
    else:
        status = "✅ Likely Normal - No significant findings"

    # Create visualization
    fig, ax = plt.subplots(figsize=(10, 6))
    diseases = list(results.keys())
    confidences = list(results.values())
    colors = ['green', 'red', 'green', 'orange']

    bars = ax.barh(diseases, confidences, color=colors, alpha=0.7)
    ax.set_xlabel('Confidence Score', fontsize=12)
    ax.set_title('Prediction Results', fontsize=14, fontweight='bold')
    ax.set_xlim(0, 1)

    # Add value labels
    for i, (bar, conf) in enumerate(zip(bars, confidences)):
        ax.text(conf + 0.02, i, f'{conf*100:.1f}%',
                va='center', fontweight='bold')

    plt.tight_layout()

    return status, results, fig

In [10]:
# Create Gradio Interface
print("🎨 Creating Gradio interface...")

demo = gr.Interface(
    fn=predict_disease,
    inputs=gr.Image(label="Upload Chest X-ray Image"),
    outputs=[
        gr.Textbox(label="Diagnosis", lines=2),
        gr.Label(label="Confidence Scores", num_top_classes=4),
        gr.Plot(label="Detailed Results")
    ],
    title="🫁 Chest X-ray Disease Detection System",
    description="""### AI-Powered Chest X-ray Analysis

**Detects:** Pneumonia and COVID-19

**How to use:**
1. Upload a chest X-ray image (JPG/PNG)
2. Wait 2-3 seconds for analysis
3. View results with confidence scores

⚠️ **Medical Disclaimer:** This is an educational tool. Always consult healthcare professionals for medical diagnosis.

**Technology:** DenseNet-121 with Transfer Learning | **Training:** Google Colab GPU
    """,
    examples=None,  # You can add example images here
    theme=gr.themes.Soft(),
    allow_flagging="never"
)

print("✅ Gradio interface ready!")

🎨 Creating Gradio interface...
✅ Gradio interface ready!


## 🚀 PART 3: Launch the App!


In [11]:
# Launch Gradio app
print("\n" + "="*70)
print("🚀 LAUNCHING GRADIO WEB APP")
print("="*70)
print("\n📱 Your app will be available at:")
print("   1. Local URL (in Colab)")
print("   2. Public URL (share with anyone!)\n")

demo.launch(
    share=True,  # Creates public link!
    debug=False
)


🚀 LAUNCHING GRADIO WEB APP

📱 Your app will be available at:
   1. Local URL (in Colab)
   2. Public URL (share with anyone!)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://74651b5af039c1c3fc.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## 🎉 SUCCESS!

### ✅ What You've Accomplished:

1. ✅ Loaded pre-trained AI models (Pneumonia & COVID-19)
2. ✅ Created beautiful Gradio web app
3. ✅ Got shareable public link!

### 📱 Share Your App:

- Copy the **public URL** from above
- Share with friends, colleagues, or anyone interested
- Works on mobile phones too!
- Link expires in 72 hours (re-run to get new link)

### 💾 Model Locations:

Your pre-trained models are in Google Drive at:

- `MyDrive/Zikriya/models/pneumonia_model_best.h5`
- `MyDrive/Zikriya/models/covid_model_best.h5`
